# RNN

Note that here testing is done on .wav files. For testing on spectrogram , the readDir function can be changed accordingly. However here too, the MFCC features are calculated from decibel power spectrum only.

In [1]:
from numpy import array
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import TimeDistributed
from keras.layers import LSTM
import pandas as pd

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import log, dot, e
import librosa
from numpy.random import rand
import tensorflow as tf
from tensorflow import keras

import tensorflow.keras as keras

In [ ]:
def splitData(X, t, testFraction=0.2, randomize = False):
    """
    Split the data randomly into training and test sets
    Use numpy functions only
    Inputs:
        X: (np array of len Nclips) input feature vectors
        t: (np array of len Nclips) targets; one hot vectors
        testFraction: (float) Nclips_test = testFraction * Nclips
    Outputs:
        X_train: training set
        X_test: test set
        t_train: training labels
        t_test: test labels
    """


    ### WRITE YOUR CODE HERE - 5 MARKS

    
    test_samples = (int) (len(X)*testFraction )
    
    temp=np.arange(len(X))
    np.random.shuffle(temp)
    
    
    temp1 = temp[0: test_samples]
    temp2 = temp[test_samples: len(X)]
    
   
    
    X_test = np.reshape(X[temp1[0]],(-1,1))
    X_test = np.transpose(X_test)
    
    t_test = np.reshape(t[temp1[0]],(-1,1))
    t_test = np.transpose(t_test)
    for x in range(1,len(temp1)):
        i=temp1[x]
        a=np.reshape(X[i],(-1,1))
        a=np.transpose(a)
        b=np.reshape(t[i],(-1,1))
        b=np.transpose(b)
        X_test = np.concatenate((X_test,a))
        t_test = np.concatenate((t_test,b))
        
    X_train = np.reshape(X[temp2[0]],(-1,1))
    X_train = np.transpose(X_train)
    
    t_train = np.reshape(t[temp2[0]],(-1,1))
    t_train = np.transpose(t_train)
    for x in range(1,len(temp2)):
        i=temp2[x]
        a=np.reshape(X[i],(-1,1))
        a=np.transpose(a)
        b=np.reshape(t[i],(-1,1))
        b=np.transpose(b)
        X_train = np.concatenate((X_train,a))
        t_train = np.concatenate((t_train,b))
        
    return X_train, t_train, X_test, t_test

In [83]:
def predict_framewise(model,x_test):
        '''
        Framewise classification (speech or music)

        '''
        
        
        lis=[]   

        
        for i in range(0,len(x_test)):
            k=0
            
            temp =np.ones((1,2))
            t2=len(x_test[i][0])
            x=x_test[i][0]
            print(x_test[i][0].shape)
            t1= np.ones( (313-len(x_test[i][0]), 60  ))
            x=np.concatenate((x,t1))
            #print(x_test[i].shape)
            x= x[np.newaxis,...]
            feat_4 = model.model.predict(x)
            
            p=np.ones((313,2))
                
            for j in range(0,len(feat_4[0])):   
                
                if(feat_4[0][j][0] > 0.5):
                    temp = np.concatenate( (temp, np.transpose(np.reshape([1,0] , (-1,1) ) ) ) ) 
                else:
                    temp = np.concatenate((temp, np.transpose(np.reshape([0,1] , (-1,1) ) ) ) ) 
                    
            #print(temp.shape)
                    
            temp = temp[1:t2+1]
            
                
            
            lis.append(temp)

        ### WRITE YOUR CODE HERE - 5 MARKS

        
        
        return lis

In [5]:
def predict_aggregate(y_pred_framewise):
        '''
        Aggregate frames to give a single class label (music or speech) to the entire audio file

            
        '''
       

        
        y_hat= np.ones((1,2))
        for i in range (0,len(y_pred_framewise)):
            
            zero=0
            one=0
            t=y_pred_framewise[i]
            for j in range(0, len(t)):
                
                if(t[j][0] == 1):
                    one+=1
                else:
                    zero+=1
            if(one>zero):
                y_hat= np.concatenate((y_hat, np.transpose(np.reshape([1,0] , (-1,1) ) ) )) 
                
            else:
                y_hat= np.concatenate((y_hat, np.transpose(np.reshape([0,1] , (-1,1) ) ) ))  

        ### WRITE YOUR CODE HERE - 5 MARKS

        y_hat=y_hat[1:]
        return y_hat

In [6]:
from glob import glob 
def readDir(dirname, Fs = 16000):
    
    '''
   This function divides input clip into parts of 10 seconds and outputs concatenated samples from all 10 seconds audio clips
    '''  


    no_samp_in_10_sec = 10* Fs
    files = glob(dirname + '/*.wav')
    x= np.ones((1,no_samp_in_10_sec))
    
    
    
    
    
    for f in files:
    
        
        samples = load_audio(f,Fs)
       
        if(len(samples) > no_samp_in_10_sec):
            i=0
            while(i + no_samp_in_10_sec <= len(samples) ):
                temp= samples[i : i +  no_samp_in_10_sec].reshape(-1,1)
                temp=np.transpose(temp)
                x= np.concatenate((x,temp))
                i=i+no_samp_in_10_sec
        
        
        else:
            temp=np.append(samples , np.zeros(no_samp_in_10_sec - len(samples) ) )
            temp=np.reshape(temp,(-1,1))
            temp = np.transpose(temp)
            x=np.concatenate((x,temp))
    
    #print(k)
    
    return x[1:]


    return x 

In [7]:
def load_audio(filename, Fs = 16000):
    '''
    Inputs: 
        filename: (str) filename
        Fs: (int) sampling rate
    Output: 
        x: 1D np array 
    '''
    

    

    x,sr=librosa.load(filename,sr=Fs)
    x=np.array(x)
    
    return x

In [8]:
def prepare_datasets(X,y,test_size, validation_size):

    
    X_train,  y_train, X_test, y_test = splitData(X, y, test_size)
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    #X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=validation_size)
    X_train, y_train, X_validation, y_validation = splitData(X_train,y_train,0.1)
    return X_train, X_validation, X_test, y_train, y_validation, y_test


In [9]:
def audio2mfcc(x, n_mfcc = 20, Fs = 16000):
    
    '''
    Compute Mel-frequency cepstral coefficients (MFCCs)
    This function finds mfcc features and thier 1st and 2nd order differences, from the log power spectrum of the given audio clip
    '''

    ### WRITE YOUR CODE HERE - 3 MARKS
    
   
    lis=[]
    if(len(x.shape) <=1):
        x= np.reshape(x, (1,len(x))) 
    
    for i in range(0, len(x)):
        
        
        n_fft = 1024
        hop_length = 512
        win_length = 1024
        X = np.abs(librosa.stft(x[i], n_fft = n_fft, hop_length = hop_length, win_length = win_length, window='hann'))
        X = librosa.power_to_db(X,ref=np.max)
        
        temp=pow(10,X/10)
        mel_spectrogram = librosa.feature.melspectrogram(y='none',S=temp, sr=16000, n_fft=n_fft, hop_length=hop_length, n_mels=20,win_length = win_length, window='hann')
        log_mel_spectrogram = librosa.power_to_db(mel_spectrogram)
        
        
        mfccs = librosa.feature.mfcc(y='none',S=log_mel_spectrogram, n_mfcc=20, sr=16000,hop_length = hop_length, win_length = win_length ,n_fft=n_fft,window='hann')
        
        
        
        
        delta_mfccs = librosa.feature.delta(mfccs)
        delta2_mfccs = librosa.feature.delta(mfccs, order=2)
        mfccs=np.concatenate((mfccs,delta_mfccs))
        mfccs=np.concatenate((mfccs,delta2_mfccs))
        mfccs=np.array(mfccs)
        mfccs=np.transpose(mfccs)
        lis.append(mfccs)
    ### WRITE YOUR CODE HERE - 3 MARKS
    
    X=np.array(lis)

     

    return X 

In [1]:
class Classifier: 
    '''
    Create a linear classifier to classify each frame
    '''
    def __init__(self):
        self.W=np.random.rand(20,)
    
    def sigmoid(self, z): return 1 / (1 + e**(-z))
    

   
    
    def rnn( self,X_train, y_train, input_shape):
        
        model = Sequential()

        model.add(keras.layers.LSTM(64, input_shape=input_shape, return_sequences=True))
        model.add(keras.layers.LSTM(64,  return_sequences=True))


        model.add(TimeDistributed(
        Dense(64, activation='relu')
        ))
        model.add(Dropout(.3))

        model.add(TimeDistributed(
        Dense(32, activation='relu')
        ))
        model.add(Dropout(.3))


       
        model.add(TimeDistributed(
        Dense(2, activation='softmax')
        ))

        model.compile(optimizer='adam', loss=tf.losses.CategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
        model.fit(X_train, y_train, batch_size=32, epochs=100)

    
        
        self.model=model
    
    def save_model(self, save_path):
        '''
        Save the trained model on local disk
        Input:
            save_path: location at which model is to be saved
        Output:
            None
            
        '''
        ## Assuming save_path contains the file name too. If save_path contains only directory
        ## name , uncomment the below line to save file as data.npy
        
        #save_path = save_path +'/data'
        
        data=self.W
        np.save(save_path, data)
        
        ### WRITE YOUR CODE HERE - 0 MARKS
            
        return
    
    def load_model(self, load_path):
        '''
        Save the trained model on local disk
        Input:
            load_path: location from which model is to be loaded
        Output:
            None
        '''
        
        
        ### WRITE YOUR CODE HERE - 0 MARKS
        
        ## Assuming load_path also contains the name of file which has to be loaded. If load_path only contains the
        ## directory name, uncomment the below line and replace data.npy with file name.
        
        #load_path = load_path +'/data.npy'
        
        self.W = np.load(load_path)
            
        return


    
    def predict_framewise(self,x_test):
        '''
        Framewise classification (speech or music)
        Input:
            x_test: test set
        Output:
            y_pred_framewise = framewise prediction
        '''
        
        lis = []
        
        if(len(x_test.shape) ==2 ):
            x_test = np.reshape(x_test,(1,len(x_test) , len(x_test[0]) ))
            

        
        for i in range(0,len(x_test)):
            k=0
            t=np.transpose(x_test[i])
            temp =np.ones((1,2))
            

            feat_4 = self.model.predict(t)
                
                
            for j in range(0,len(feat_4)):   
                
                if(feat_4[j][0] > 0.5):
                    temp = np.concatenate( (temp, np.transpose(np.reshape([1,0] , (-1,1) ) ) ) ) 
                else:
                    temp = np.concatenate((temp, np.transpose(np.reshape([0,1] , (-1,1) ) ) ) ) 
                    
            #print(temp.shape)
                    
            temp = temp[1:]
            temp=np.transpose(temp)
                
            
            lis.append(temp)

        ### WRITE YOUR CODE HERE - 5 MARKS

        y_pred_framewise= np.array(lis)
        
        return y_pred_framewise 
    
    def predict_aggregate(self,y_pred_framewise):
        '''
        Aggregate frames to give a single class label (music or speech) to the entire audio file
        Input:
            y_pred_framewise = framewise prediction
        Output:
            y_hat = frame aggregate (one-hot vectors)
            
        '''
        if(len(y_pred_framewise.shape) ==2 ):
            y_pred_framewise = np.reshape(y_pred_framewise, (1,2,len(y_pred_framewise[0])))        

        
        y_hat= np.ones((1,2))
        for i in range (0,len(y_pred_framewise)):
            
            zero=0
            one=0
            t=np.transpose(y_pred_framewise[i])
            for j in range(0, len(t)):
                
                if(t[j][0] == 1):
                    one+=1
                else:
                    zero+=1
            if(one>zero):
                y_hat= np.concatenate((y_hat, np.transpose(np.reshape([1,0] , (-1,1) ) ) )) 
                
            else:
                y_hat= np.concatenate((y_hat, np.transpose(np.reshape([0,1] , (-1,1) ) ) ))  

        ### WRITE YOUR CODE HERE - 5 MARKS

        y_hat=y_hat[1:]
        return y_hat

In [11]:
def computeCM(y, y_hat):
    '''
    Compute confusion matrix to evaluate your model
    Inputs:
        y = labels 
        y_hat = predicted output
    Output:
        confusion matrix: confusion matrix
    '''

    ### WRITE YOUR CODE HERE - 5 MARKS


    metrics = np.array([[0,0], [0,0]])

    
    for i in range(0,len(y)):
        if(y[i][0] == 1 and y_hat[i][0] == 1):
            metrics[0][0]+=1
        elif(y[i][0] == 1 and y_hat[i][0] == 0):
            metrics[0][1] +=1
        elif(y[i][0] == 0 and y_hat[i][0] == 1):
            metrics[1][0] +=1
        else:
            metrics[1][1]+=1

    confusion_matrix = metrics
    return confusion_matrix 

In [60]:
# silence vs audio training
x_silence = readDir('C:/Users/HP/Documents/test/no-audio', 16000) 
x_speech =  readDir('C:/Users/HP/Documents/test/audio', 16000)


X = np.concatenate((x_silence, x_speech))
y_silence = np.load('file_path') #File  Containing framewise labels of all samples
y_speech = np.load('file_path') #File  Containing framewise labels of all samples
Y = np.concatenate((y_silence, y_speech))

X=audio2mfcc(X,20, 16000)

lis=[]
for i in range(0,len(Y)):
    lis.append( np.array(  [[Y[i][0], Y[i][1]]]* len(X[1]) )  )
Y=np.array(lis)

X_train, X_validation, X_test, y_train, y_validation, y_test = prepare_datasets(X,Y,0.25, 0.2)
input_shape = (X.shape[1], X.shape[2])


model1=Classifier()
model1.rnn(X, Y, input_shape)

Epoch 1/100
7/7 [==============================] - 4s 221ms/step - loss: 0.7405 - accuracy: 0.5066
Epoch 2/100
7/7 [==============================] - 2s 220ms/step - loss: 0.7027 - accuracy: 0.5472
Epoch 3/100
7/7 [==============================] - 2s 218ms/step - loss: 0.6940 - accuracy: 0.5489
Epoch 4/100
7/7 [==============================] - 2s 216ms/step - loss: 0.6928 - accuracy: 0.5490
Epoch 5/100
7/7 [==============================] - 1s 214ms/step - loss: 0.6920 - accuracy: 0.5497
Epoch 6/100
7/7 [==============================] - 2s 222ms/step - loss: 0.6918 - accuracy: 0.5501
Epoch 7/100
7/7 [==============================] - 1s 212ms/step - loss: 0.6910 - accuracy: 0.5530
Epoch 8/100
7/7 [==============================] - 2s 220ms/step - loss: 0.6894 - accuracy: 0.5592
Epoch 9/100
7/7 [==============================] - 2s 224ms/step - loss: 0.6869 - accuracy: 0.5700
Epoch 10/100
7/7 [==============================] - 2s 220ms/step - loss: 0.6826 - accuracy: 0.5847
Epoch 11/

7/7 [==============================] - 2s 293ms/step - loss: 0.4525 - accuracy: 0.8594
Epoch 84/100
7/7 [==============================] - 5s 729ms/step - loss: 0.4675 - accuracy: 0.8442
Epoch 85/100
7/7 [==============================] - 3s 473ms/step - loss: 0.4556 - accuracy: 0.8555
Epoch 86/100
7/7 [==============================] - 3s 440ms/step - loss: 0.4654 - accuracy: 0.8463
Epoch 87/100
7/7 [==============================] - 4s 476ms/step - loss: 0.4646 - accuracy: 0.8474
Epoch 88/100
7/7 [==============================] - 8s 1s/step - loss: 0.4530 - accuracy: 0.8587
Epoch 89/100
7/7 [==============================] - 4s 615ms/step - loss: 0.4707 - accuracy: 0.8409
Epoch 90/100
7/7 [==============================] - 3s 369ms/step - loss: 0.4634 - accuracy: 0.8480
Epoch 91/100
7/7 [==============================] - 3s 468ms/step - loss: 0.4554 - accuracy: 0.8564
Epoch 92/100
7/7 [==============================] - 4s 566ms/step - loss: 0.4501 - accuracy: 0.8613
Epoch 93/100
7/7

In [61]:
# speech vs music training
x_speech = readDir('C:/Users/HP/Documents/test/speech', 16000)
x_music = readDir('C:/Users/HP/Documents/test/music', 16000) 
X = np.concatenate((x_speech, x_music))
y_speech = np.array([[1,0]]*len(x_speech))
y_music= np.array([[0,1]]*len(x_music))
Y = np.concatenate((y_speech, y_music))

X=audio2mfcc(X,20, 16000)

lis=[]
for i in range(0,len(Y)):
    lis.append( np.array(  [[Y[i][0], Y[i][1]]]* len(X[1]) )  )
Y=np.array(lis)



X_train, X_validation, X_test, y_train, y_validation, y_test = prepare_datasets(X,Y,0.25, 0.2)
input_shape = (X_train.shape[1], X_train.shape[2])

#X=X_train

model2=Classifier()
model2.rnn(X_train, y_train,input_shape)

Epoch 1/100
4/4 [==============================] - 3s 172ms/step - loss: 0.7868 - accuracy: 0.5049
Epoch 2/100
4/4 [==============================] - 1s 169ms/step - loss: 0.7911 - accuracy: 0.5003
Epoch 3/100
4/4 [==============================] - 1s 177ms/step - loss: 0.7111 - accuracy: 0.5915
Epoch 4/100
4/4 [==============================] - 1s 203ms/step - loss: 0.6907 - accuracy: 0.6076
Epoch 5/100
4/4 [==============================] - 1s 222ms/step - loss: 0.6743 - accuracy: 0.6311
Epoch 6/100
4/4 [==============================] - 1s 178ms/step - loss: 0.6766 - accuracy: 0.6328
Epoch 7/100
4/4 [==============================] - 1s 178ms/step - loss: 0.6721 - accuracy: 0.6363
Epoch 8/100
4/4 [==============================] - 1s 173ms/step - loss: 0.6741 - accuracy: 0.6360
Epoch 9/100
4/4 [==============================] - 1s 172ms/step - loss: 0.6781 - accuracy: 0.6354
Epoch 10/100
4/4 [==============================] - 1s 171ms/step - loss: 0.6772 - accuracy: 0.6362
Epoch 11/

4/4 [==============================] - 2s 625ms/step - loss: 0.6667 - accuracy: 0.6370
Epoch 84/100
4/4 [==============================] - 2s 374ms/step - loss: 0.6644 - accuracy: 0.6379
Epoch 85/100
4/4 [==============================] - 2s 426ms/step - loss: 0.6650 - accuracy: 0.6371
Epoch 86/100
4/4 [==============================] - 3s 800ms/step - loss: 0.6649 - accuracy: 0.6378
Epoch 87/100
4/4 [==============================] - 2s 331ms/step - loss: 0.6632 - accuracy: 0.6386
Epoch 88/100
4/4 [==============================] - 1s 309ms/step - loss: 0.6613 - accuracy: 0.6378
Epoch 89/100
4/4 [==============================] - 1s 251ms/step - loss: 0.6624 - accuracy: 0.6375
Epoch 90/100
4/4 [==============================] - 1s 270ms/step - loss: 0.6584 - accuracy: 0.6388
Epoch 91/100
4/4 [==============================] - 1s 250ms/step - loss: 0.6549 - accuracy: 0.6393
Epoch 92/100
4/4 [==============================] - 1s 242ms/step - loss: 0.6515 - accuracy: 0.6434
Epoch 93/100


In [62]:
#  audio vs no audio prediction

data = readDir('C:/Users/HP/Documents/test/unknown/', 16000) # This will work for single file in folder. For multiple files, WE can run this in a loop for every unknown audio file in folder
data=audio2mfcc(data)
pred= model1.model.predict(data)


p=np.ones((len(X[1]),2))

for i in range(0, len(X[1])):
    if(pred[0][i][1]>0.5):
        p[i][0]=0
        p[i][1]=1
    else:
        p[i][0]=1
        p[i][1]=0

In [63]:
#Time label prediction

k=0
j=0
sil=np.array([[0.0,0.0],[0.0,0.0],[0.0,0.0],[0.0,0.0],[0.0,0.0],[0.0,0.0]])
sp=np.array([[0.0,0.0],[0.0,0.0],[0.0,0.0],[0.0,0.0],[0.0,0.0],[0.0,0.0]])

def check(x):
    sum=0
    for i in range(0,len(x)):
        sum=sum+x[i][0]
    if(sum>=10):
        return 1
    else:
        return 0
    
    
    
prev=1

for i in range(0,len(p)-16):
    x=p[i:i+16]
    a=check(x)
    #print(a)
    if(a==1):
        if(prev==1):
            sil[j][1]=librosa.frames_to_time(i+22, sr=16000, hop_length=512, n_fft=1024)
        else:
            temp=i
            while(p[i][0]!=1):
                i+=1
            sp[k][1]=librosa.frames_to_time(i, sr=16000, hop_length=512, n_fft=1024)
            sil[j][0]=librosa.frames_to_time(i, sr=16000, hop_length=512, n_fft=1024)
            sil[j][1]=librosa.frames_to_time(temp+22, sr=16000, hop_length=512, n_fft=1024)
            k+=1
        
    
    else:
        if(prev==1):
            temp=i
            while(p[i][1]!=1):
                i+=1
            sil[j][1]=librosa.frames_to_time(i, sr=16000, hop_length=512, n_fft=1024)
            sp[k][0]=librosa.frames_to_time(i, sr=16000, hop_length=512, n_fft=1024)
            sp[k][1]=librosa.frames_to_time(temp+22, sr=16000, hop_length=512, n_fft=1024)
            j+=1
        
        else:
            sp[k][1]=librosa.frames_to_time(i+22, sr=16000, hop_length=512, n_fft=1024)
    prev=a      

    
sp  

array([[8.32 , 8.672],
       [0.   , 0.   ],
       [0.   , 0.   ],
       [0.   , 0.   ],
       [0.   , 0.   ],
       [0.   , 0.   ]])

In [64]:
#Extracting audio portions

lis=[]
for i in range(len(sp)):
    if(sp[i][0]==0 and sp[i][1]==0):
        continue
    else:
        lis.append(np.array([sp[i][0],sp[i][1]]))
sp=np.array(lis)


lis=[]
for i in range(0,len(sp)):
    start= librosa.time_to_frames(sp[i][0], sr=16000, hop_length=512, n_fft=1024)
    end = librosa.time_to_frames(sp[i][1], sr=16000, hop_length=512, n_fft=1024)
    print(start,end)
    temp= np.array(data[0][start:end])
    temp = temp[ np.newaxis, ...]
    lis.append(temp)

259 270


In [84]:
frame_pred= predict_framewise(model2,lis)
#print(len(frame_pred))
agg=predict_aggregate(frame_pred)  

for i in range(len(sp)):
    print(sp[i][0],sp[i][1], 'speech' if (agg[i][0]==1) else  'music')

(11, 60)
8.32 8.672 music
